In [ ]:
%matplotlib inline

In [ ]:
from clawpack.geoclaw import dtopotools
import numpy as np
import matplotlib.pyplot as pl

## Comninou & Dundurs model (Okada model for triangular faults)

In this notebook, we will check the results for the newly written Comninou & Dundurs model in `dtopotools`, which computes the elastic half-space deformations due to triangular faults.

This code currently is stored in: 
   
   * the fork `github.com/dsrim/geoclaw` in the branch `triangular_angular`.

First, we load a `SiftFault` to import a rectangular fault

In [ ]:
fault = dtopotools.SiftFault({'acszb2':1.})
subfault0 = fault.subfaults[0]

Then, we first compute the seafloor deformation using the Okada model

In [ ]:
# Find a sufficiently large rectangle containing the fault
lims = fault.containing_rect()
slip = 1.

# setting limits to compute the deformations
x0 = lims[0]-1.
x1 = lims[1]+1.

y0 = lims[2]-1.
y1 = lims[3]+1.

x = np.linspace(x0,x1,100)
y = np.linspace(y0,y1,100)
fault.subfaults[0].slip = slip
dtopo0 = fault.create_dtopography(x,y,times=[0.])

In [ ]:
dtopo0.plot_dZ_colors(0.);

The *strike*, *dip*, *rake*, *depth*, *width*, and *length* of this fault is given as:

In [ ]:
print('strike = ' + '{:6.2f}'.format(subfault0.strike))
print('dip = ' + '{:6.2f}'.format(subfault0.dip))
print('rake = ' + '{:6.2f}'.format(subfault0.rake))
print('depth = ' + '{:6.2f}'.format(subfault0.depth))
print('width = ' + '{:6.2f}'.format(subfault0.width))
print('length = ' + '{:6.2f}'.format(subfault0.length))

The long-lat coordinates of the four vertices of the rectangle is stored in the variable `subfault0.corners`:

In [ ]:
print('four corners = ')
print(np.array(subfault0.corners))

### Setting up a triangular fault

A triangular fault is designated by supplying the long-lat-depth coordinates of the three corners to the subfault `subfault.set_corners` routine.

In [ ]:
rect_corners = subfault0.corners

subfault1 = dtopotools.SubFault()
subfault2 = dtopotools.SubFault()

subfault1.set_corners([rect_corners[0],rect_corners[1],rect_corners[2]])
subfault2.set_corners([rect_corners[0],rect_corners[2],rect_corners[3]])

In [ ]:
print('corners for subfault 1 =')
print(np.array(subfault1.corners))
print(' ')
print('corners for subfault 2 =')
print(np.array(subfault2.corners))

In [ ]:
print('subfault0 strike: ' + '{:6.2f}'.format( subfault1.strike))
print('subfault1 strike: ' + '{:6.2f}'.format( subfault2.strike))
print('strike difference: ' + '{:6.2f}'.format( subfault2.strike - subfault1.strike))
print('---')
print('subfault0 dip: ' + '{:6.2f}'.format( subfault1.dip))
print('subfault1 dip: ' + '{:6.2f}'.format( subfault2.dip))
print('---')
print('subfault0 depth: ' + '{:6.2f}'.format( subfault1.depth))
print('subfault1 depth: ' + '{:6.2f}'.format( subfault2.depth))
print('---')
print('subfault0 rake: ' + '{:6.2f}'.format( subfault1.rake))
print('subfault1 rake: ' + '{:6.2f}'.format( subfault2.rake))
print('---')
print('subfault0 unit slip vector: ' + str(subfault1._get_unit_slip_vector()))
print('subfault1 unit slip vector: ' + str(subfault2._get_unit_slip_vector()))

In [ ]:
rake = 90.

subfault1.rake = rake
subfault2.rake = rake

subfault1.slip = slip
subfault2.slip = slip

# put two subfaults into a fault
fault1 = dtopotools.Fault()
fault1.subfaults = []
fault1.subfaults.append(subfault1)
fault1.subfaults.append(subfault2)

# compute deformation
dtopo1 = fault1.create_dtopography(x,y,times=[0.]);

print('deformation from two triangular faults to the left, Okada to the right')
fig,ax = pl.subplots(nrows=1, ncols=2, figsize=(13,5))
dtopo1.plot_dZ_colors(0.,axes = ax[0], cmax_dZ=0.4, dZ_interval=0.05);
dtopo0.plot_dZ_colors(0.,axes = ax[1], cmax_dZ=0.4, dZ_interval=0.05);

In [ ]:
ddZ = dtopo1.dZ[-1,:,:] - dtopo0.dZ[-1,:,:]
print('min /max of ddZ: ', (ddZ.min(), ddZ.max()))

cm=pl.get_cmap('seismic')
fault1.plot_subfaults();
pl.contourf(x,y,ddZ,np.linspace(-.04,.04,10),cmap=cm)
pl.xlim(164,167)
pl.ylim(54.,56)
pl.colorbar();

In [ ]:
print('Plot of the two triangular subfaults')
fault1.plot_subfaults();

In [ ]:
print('mean error = ' + '{:6.4f}'.format(np.mean(np.abs(dtopo1.dZ - dtopo0.dZ))))

In [ ]:
f,ax = pl.subplots(nrows=1,ncols=2,figsize=(13,5));

dtopoT1 = subfault1.okada(x,y)
dtopoT2 = subfault2.okada(x,y)

print('deformations from two triangles separately')
dtopoT1.plot_dZ_colors(0.,axes=ax[0]);
dtopoT2.plot_dZ_colors(0.,axes=ax[1]);

## TODO

* bulk set a lot of triangles through an attribute function for `Fault` object
* read / write, especially for slab format

## Try subdividing fault to get better agreement?

In [ ]:
fault2 = dtopotools.SubdividedPlaneFault(subfault0, nstrike=8, ndip=4)

In [ ]:
fault2.plot_subfaults()

In [ ]:
# compute deformation
dtopo2 = fault2.create_dtopography(x,y,times=[0.]);

In [ ]:
fault3 = dtopotools.Fault()
fault3.subfaults = []
for s in fault2.subfaults:
    rect_corners = s.corners
    subfault1 = dtopotools.SubFault()
    subfault2 = dtopotools.SubFault()
    subfault1.slip = slip
    subfault2.slip = slip
    subfault1.set_corners([rect_corners[0],rect_corners[1],rect_corners[2]])
    subfault2.set_corners([rect_corners[0],rect_corners[2],rect_corners[3]])
    #print(subfault1.latitude, subfault1.longitude)
    fault3.subfaults.append(subfault1)
    fault3.subfaults.append(subfault2)

In [ ]:
fault3.plot_subfaults()

In [ ]:
# compute deformation
dtopo3 = fault3.create_dtopography(x,y,times=[0.]);

print('deformation from triangulation to the left, Okada to the right')
fig,ax = pl.subplots(nrows=1, ncols=2, figsize=(13,5))
dtopo3.plot_dZ_colors(0.,axes = ax[0], cmax_dZ=0.4, dZ_interval=0.05);
dtopo2.plot_dZ_colors(0.,axes = ax[1], cmax_dZ=0.4, dZ_interval=0.05);

In [ ]:
ddZ = dtopo3.dZ[-1,:,:] - dtopo2.dZ[-1,:,:]
print('min /max of ddZ: ', (ddZ.min(), ddZ.max()))

cm=pl.get_cmap('seismic')
fault3.plot_subfaults();
pl.contourf(x,y,ddZ,np.linspace(-.04,.04,10),cmap=cm)
pl.xlim(164,167)
pl.ylim(54.,56)
pl.colorbar();